# install 하기

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install sentencepiece

# import

In [ ]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy
import numpy as np

from transformers import ElectraModel, ElectraTokenizer
from transformers import AutoModel, ElectraTokenizer


# 모델 구축

## 기본설정

In [ ]:

PADDING_TOKEN = 1
S_OPEN_TOKEN = 0
S_CLOSE_TOKEN = 2

do_eval=True

# 모델을 학습 할 때 저장 되는 파일 경로
category_extraction_model_path = '/content/drive/MyDrive/korean_baseline/saved_model/category_extraction/'
polarity_classification_model_path = '/content/drive/MyDrive/korean_baseline/saved_model/polarity_classification/'


# 저장된 모델 Weight 파일 경로
test_category_extraction_model_path = '/content/drive/MyDrive/korean_baseline/saved_model/category_extraction/category_sample.pt'
test_polarity_classification_model_path = '/content/drive/MyDrive/korean_baseline/saved_model/polarity_classification/polarity_sample.pt'

# 데이터 파일 경로
train_cate_data_path = '/content/drive/MyDrive/train_sample(category).jsonl'
train_pola_data_path = '/content/drive/MyDrive/train_sample(polarity).jsonl'
test_data_path = '/content/drive/MyDrive/test_sample.jsonl'

max_len_elec = 256
max_len_debe = 256
max_len_robe = 514

# colab pro 환경에서 RoBERTa를 돌리게 될 경우 batch_size 수정 요망 ( Out of Memory 이슈 )
batch_size = 32

#ELECTRA
base_model_elec = 'kykim/electra-kor-base'
#RoBERTa
base_model_roberta = 'xlm-roberta-base'
#DeBERTa
base_model_deberta = "lighthouse/mdeberta-v3-base-kor-further"

learning_rate = 3e-6
eps = 1e-8
num_train_epochs = 30

classifier_hidden_size_base = 768
classifier_hidden_size_down = 384   # hidden_size down
classifier_hidden_size_up = 1000    # hidden_size up

classifier_dropout_prob_base = 0.1  # dropout = 0.1
classifier_dropout_prob_up = 0.5    # dropout = 0.5

# 카테고리의 수 = 25개
entity_property_pair = [
     '패키지/구성품#다양성','본품#인지도','브랜드#디자인',
     '패키지/구성품#편의성','제품 전체#디자인', '제품 전체#품질',
     '패키지/구성품#품질','패키지/구성품#일반','본품#일반',
     '패키지/구성품#디자인','본품#편의성','브랜드#품질',
     '브랜드#인지도','본품#다양성','본품#디자인',
     '제품 전체#다양성','본품#품질','제품 전체#인지도',
     '패키지/구성품#가격','본품#가격','제품 전체#가격',
     '브랜드#가격','브랜드#일반','제품 전체#일반','제품 전체#편의성'
     ]

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

In [ ]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [ ]:
def tokenize_and_align_labels(tokenizer, form, annotations, max_len):

    entity_property_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }
    polarity_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }

    for pair in entity_property_pair:
        isPairInOpinion = False
        if pd.isna(form):
            break
        tokenized_data = tokenizer(form, pair, padding='max_length', max_length=max_len, truncation=True)
        for annotation in annotations:
            entity_property = annotation[0]
            polarity = annotation[2]

            if polarity == '------------':
                continue

            if entity_property == pair:
                entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
                entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                entity_property_data_dict['label'].append(tf_name_to_id['True'])

                polarity_data_dict['input_ids'].append(tokenized_data['input_ids'])
                polarity_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                polarity_data_dict['label'].append(polarity_name_to_id[polarity])

                isPairInOpinion = True
                break

        if isPairInOpinion is False:
            entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
            entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
            entity_property_data_dict['label'].append(tf_name_to_id['False'])

    return entity_property_data_dict, polarity_data_dict


def get_dataset(raw_data, tokenizer, max_len):
    input_ids_list = []
    attention_mask_list = []
    token_labels_list = []

    polarity_input_ids_list = []
    polarity_attention_mask_list = []
    polarity_token_labels_list = []

    for utterance in raw_data:
        entity_property_data_dict, polarity_data_dict = tokenize_and_align_labels(tokenizer, utterance['sentence_form'], utterance['annotation'], max_len)
        input_ids_list.extend(entity_property_data_dict['input_ids'])
        attention_mask_list.extend(entity_property_data_dict['attention_mask'])
        token_labels_list.extend(entity_property_data_dict['label'])

        polarity_input_ids_list.extend(polarity_data_dict['input_ids'])
        polarity_attention_mask_list.extend(polarity_data_dict['attention_mask'])
        polarity_token_labels_list.extend(polarity_data_dict['label'])

    return TensorDataset(torch.tensor(input_ids_list), torch.tensor(attention_mask_list),
                         torch.tensor(token_labels_list)), TensorDataset(torch.tensor(polarity_input_ids_list), torch.tensor(polarity_attention_mask_list),
                         torch.tensor(polarity_token_labels_list))



In [ ]:
def evaluation(y_true, y_pred, label_len):
    count_list = [0]*label_len
    hit_list = [0]*label_len
    for i in range(len(y_true)):
        count_list[y_true[i]] += 1
        if y_true[i] == y_pred[i]:
            hit_list[y_true[i]] += 1
    acc_list = []

    for i in range(label_len):
        acc_list.append(hit_list[i]/count_list[i])

    print(count_list)
    print(hit_list)
    print(acc_list)
    print('accuracy: ', (sum(hit_list) / sum(count_list)))
    print('macro_accuracy: ', sum(acc_list) / 3)
    # print(y_true)

    y_true = list(map(int, y_true))
    y_pred = list(map(int, y_pred))

    print('f1_score: ', f1_score(y_true, y_pred, average=None))
    print('f1_score_micro: ', f1_score(y_true, y_pred, average='micro'))
    print('f1_score_macro: ', f1_score(y_true, y_pred, average='macro'))

In [ ]:
def evaluation_f1(true_data, pred_data):

    true_data_list = true_data
    pred_data_list = pred_data

    ce_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    pipeline_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    for i in range(len(true_data_list)):

        # TP, FN checking
        is_ce_found = False
        is_pipeline_found = False
        for y_ano  in true_data_list[i]['annotation']:
            y_category = y_ano[0]
            y_polarity = y_ano[2]

            for p_ano in pred_data_list[i]['annotation']:
                p_category = p_ano[0]
                p_polarity = p_ano[1]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is True:
                ce_eval['TP'] += 1
            else:
                ce_eval['FN'] += 1

            if is_pipeline_found is True:
                pipeline_eval['TP'] += 1
            else:
                pipeline_eval['FN'] += 1

            is_ce_found = False
            is_pipeline_found = False

        # FP checking
        for p_ano in pred_data_list[i]['annotation']:
            p_category = p_ano[0]
            p_polarity = p_ano[1]

            for y_ano  in true_data_list[i]['annotation']:
                y_category = y_ano[0]
                y_polarity = y_ano[2]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is False:
                ce_eval['FP'] += 1

            if is_pipeline_found is False:
                pipeline_eval['FP'] += 1
            is_ce_found = False
            is_pipeline_found = False

    ce_precision = ce_eval['TP']/(ce_eval['TP']+ce_eval['FP'])
    ce_recall = ce_eval['TP']/(ce_eval['TP']+ce_eval['FN'])

    ce_result = {
        'Precision': ce_precision,
        'Recall': ce_recall,
        'F1': 2*ce_recall*ce_precision/(ce_recall+ce_precision)
    }

    pipeline_precision = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FP'])
    pipeline_recall = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FN'])

    pipeline_result = {
        'Precision': pipeline_precision,
        'Recall': pipeline_recall,
        'F1': 2*pipeline_recall*pipeline_precision/(pipeline_recall+pipeline_precision)
    }

    return {
        'category extraction result': ce_result,
        'entire pipeline result': pipeline_result
    }

## SimpleClassifier

In [ ]:
# baseline
class SimpleClassifier_Base(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_base)
        self.dropout = nn.Dropout(classifier_dropout_prob_base)
        self.output = nn.Linear(classifier_hidden_size_base, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

# hidden_size를 1000으로 up
class SimpleClassifier_Hidden_up(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_up)
        self.dropout = nn.Dropout(classifier_dropout_prob_base)
        self.output = nn.Linear(classifier_hidden_size_up, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x        

# hidden_size를 384로 down
class SimpleClassifier_Hidden_down(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_down)
        self.dropout = nn.Dropout(classifier_dropout_prob_base)
        self.output = nn.Linear(classifier_hidden_size_down, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x        

# dropout 0.5
class SimpleClassifier_dr05(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_base)
        self.dropout = nn.Dropout(classifier_dropout_prob_up)
        self.output = nn.Linear(classifier_hidden_size_base, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

# hidden_size를 384로 down + dropout 0.5
class SimpleClassifier_Hidden_down_dr05(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_down)
        self.dropout = nn.Dropout(classifier_dropout_prob_up)
        self.output = nn.Linear(classifier_hidden_size_down, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x        

# dense_layer를 1층 더 추가 (dropout0.1)
class SimpleClassifier_Layer(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense1 = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_base//2)
        self.dense2 = nn.Linear(classifier_hidden_size_base//2, classifier_hidden_size_base//4)
        self.dropout = nn.Dropout(classifier_dropout_prob_base)
        self.output = nn.Linear(classifier_hidden_size_base//4, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        # layer 1
        x = self.dropout(x)
        x = self.dense1(x)
        
        # layer 2
        x = self.dropout(x)
        x = self.dense2(x)

        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)

        return x

# dense_layer를 1층 더 추가 + dropout 0.5
class SimpleClassifier_Layer_dr05(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense1 = nn.Linear(classifier_hidden_size_base, classifier_hidden_size_base//2)
        self.dense2 = nn.Linear(classifier_hidden_size_base//2, classifier_hidden_size_base//4)
        self.dropout = nn.Dropout(classifier_dropout_prob_up)
        self.output = nn.Linear(classifier_hidden_size_base//4, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        # layer 1
        x = self.dropout(x)
        x = self.dense1(x)
        
        # layer 2
        x = self.dropout(x)
        x = self.dense2(x)

        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)

        return x


## ELECTRA

#### baseline

In [ ]:
# category baseline
class ElectraBaseClassifier_Cate_Base(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_Base, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Base(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity baseline
class ElectraBaseClassifier_Pola_Base(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_Base, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Base(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


#### only layer 추가

In [ ]:
# category layer 추가
class ElectraBaseClassifier_Cate_Layer(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_Layer, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Layer(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity layer 추가
class ElectraBaseClassifier_Pola_Layer(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_Layer, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Layer(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


#### only dropout 0.5

In [ ]:
# category dropout 0.5
class ElectraBaseClassifier_Cate_dr05(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_dr05, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_dr05(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity dropout 0.5
class ElectraBaseClassifier_Pola_dr05(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_dr05, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_dr05(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


#### layer + dropout 0.5

In [ ]:
# category layer 추가 + dropout 0.5
class ElectraBaseClassifier_Cate_Layer_Dropout05(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_Layer_Dropout05, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Layer_dr05(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity layer 추가 + dropout 0.5
class ElectraBaseClassifier_Pola_Layer_Dropout05(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_Layer_Dropout05, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Layer_dr05(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

#### hidden_size_up

In [ ]:
# category hidden_size_up
class ElectraBaseClassifier_Cate_hiddenup(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_hiddenup, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Hidden_up(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity hidden_size_up
class ElectraBaseClassifier_Pola_hiddenup(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_hiddenup, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Hidden_up(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


#### only hidden_size_down

In [ ]:
# category hidden down
class ElectraBaseClassifier_Cate_hiddendown(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_hiddendown, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Hidden_down(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity hidden down
class ElectraBaseClassifier_Pola_hiddendown(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_hiddendown, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Hidden_down(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


#### hidden_size_down + dropout 0.5

In [ ]:
# category hidden down + dropout 0.5
class ElectraBaseClassifier_Cate_hiddendown_dr05(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Cate_hiddendown_dr05, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Hidden_down_dr05(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# polarity hidden down + dropout 0.5
class ElectraBaseClassifier_Pola_hiddendown_dr05(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier_Pola_hiddendown_dr05, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained(base_model_elec)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Hidden_down_dr05(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


## RoBERTa

In [ ]:
# category baseline(roberta)
class RobertaBaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(RobertaBaseClassifier, self).__init__()

        self.num_label = num_label
        self.roberta = AutoModel.from_pretrained(base_model_roberta) 
        self.roberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Base(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

## DeBERTa

In [ ]:
# category baseline(deberta)
class DebertaBaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(DebertaBaseClassifier, self).__init__()

        self.num_label = num_label
        self.deberta = AutoModel.from_pretrained(base_model_deberta)
        self.deberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier_Base(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.deberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

# 모델평가

## Predict

### base(ELECTRA)

In [ ]:

def predict_from_korean_form_kelec(tokenizer_kelec, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for idx, sentence in enumerate(data):
        if idx % 10 == 0:
            print(idx, "/", len(data))

        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue
        for pair in entity_property_pair:
            
            tokenized_data_kelec = tokenizer_kelec(form, pair, padding='max_length', max_length=256, truncation=True)
            input_ids_kelec = torch.tensor([tokenized_data_kelec['input_ids']]).to(device)
            attention_mask_kelec = torch.tensor([tokenized_data_kelec['attention_mask']]).to(device)

            with torch.no_grad():
                _, ce_logits = ce_model(input_ids_kelec, attention_mask_kelec)

            ce_predictions = torch.argmax(ce_logits, dim = -1)

            ce_result = tf_id_to_name[ce_predictions[0]]

            if ce_result == 'True':
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids_kelec, attention_mask_kelec)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])


    return data


### ELECTRA + Force

Force : 빈칸( '[ ]' 에 대해서 가장 높은 확률의 카테고리를 강제로 뽑아내는 방법 )

In [ ]:

def predict_from_korean_form_kelec_forcing(tokenizer_kelec, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for idx, sentence in enumerate(data):
        if idx % 10 == 0:
            print(idx, "/", len(data))

        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue


        
        tmp = []
        flag = False

        for pair in entity_property_pair:
            
            tokenized_data = tokenizer_kelec(form, pair, padding='max_length', max_length=256, truncation=True)

            input_ids = torch.tensor([tokenized_data['input_ids']]).to(device)
            attention_mask = torch.tensor([tokenized_data['attention_mask']]).to(device)


            with torch.no_grad():
                _, ce_logits = ce_model(input_ids, attention_mask)
                
                tmp.append( ce_logits[0][0] )

            ce_predictions = torch.argmax(ce_logits, dim = -1)
            
            ce_result = tf_id_to_name[ce_predictions[0]]


            if ce_result == 'True':
                flag = True
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids, attention_mask)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])
        
        if flag == False:

            tmp = torch.tensor(tmp)

            pair = entity_property_pair[torch.argmax(tmp)]

            with torch.no_grad():
                _, pc_logits = pc_model(input_ids, attention_mask)

            pc_predictions = torch.argmax(pc_logits, dim=-1)
            pc_result = polarity_id_to_name[pc_predictions[0]]

            sentence['annotation'].append([pair, pc_result])
                

    return data

### category : RoBERTa / polarity : ELECTRA

In [ ]:

def predict_from_korean_form_roberta(tokenizer_roberta, tokenizer_kelec, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for idx, sentence in enumerate(data):
        if idx % 10 == 0:
            print(idx, "/", len(data))

        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue
        for pair in entity_property_pair:
            

            tokenized_data_roberta = tokenizer_roberta(form, pair, padding='max_length', max_length=514, truncation=True)
            tokenized_data_kelec = tokenizer_kelec(form, pair, padding='max_length', max_length=256, truncation=True)

            input_ids_roberta = torch.tensor([tokenized_data_roberta['input_ids']]).to(device)
            attention_mask_roberta = torch.tensor([tokenized_data_roberta['attention_mask']]).to(device)

            input_ids_kelec = torch.tensor([tokenized_data_kelec['input_ids']]).to(device)
            attention_mask_kelec = torch.tensor([tokenized_data_kelec['attention_mask']]).to(device)

            with torch.no_grad():
                _, ce_logits = ce_model(input_ids_roberta, attention_mask_roberta)

            ce_predictions = torch.argmax(ce_logits, dim = -1)

            ce_result = tf_id_to_name[ce_predictions[0]]

            if ce_result == 'True':
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids_kelec, attention_mask_kelec)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])


    return data


### category : RoBERTa / polarity : ELECTRA + Force

Force : 빈칸( '[ ]' 에 대해서 가장 높은 확률의 카테고리를 강제로 뽑아내는 방법 )

In [ ]:
def predict_from_korean_form_roberta_forcing(tokenizer_roberta, tokenizer_kelec, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for idx, sentence in enumerate(data):
        if idx % 10 == 0:
            print(idx, "/", len(data))

        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue


        tmp = []
        flag = False

        for pair in entity_property_pair:
            
            tokenized_data_kelec = tokenizer_kelec(form, pair, padding='max_length', max_length=256, truncation=True)
            tokenized_data_roberta = tokenizer_roberta(form, pair, padding='max_length', max_length=514, truncation=True)

            input_ids_kelec = torch.tensor([tokenized_data_kelec['input_ids']]).to(device)
            attention_mask_kelec = torch.tensor([tokenized_data_kelec['attention_mask']]).to(device)

            input_ids_roberta = torch.tensor([tokenized_data_roberta['input_ids']]).to(device)
            attention_mask_roberta = torch.tensor([tokenized_data_roberta['attention_mask']]).to(device)


            with torch.no_grad():
                _, ce_logits = ce_model(input_ids_roberta, attention_mask_roberta)
                
                tmp.append( ce_logits[0][0] )

            ce_predictions = torch.argmax(ce_logits, dim = -1)
            
            ce_result = tf_id_to_name[ce_predictions[0]]


            if ce_result == 'True':
                flag = True
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids_kelec, attention_mask_kelec)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])
        
        if flag == False:

            tmp = torch.tensor(tmp)

            pair = entity_property_pair[torch.argmax(tmp)]

            with torch.no_grad():
                _, pc_logits = pc_model(input_ids_kelec, attention_mask_kelec)

            pc_predictions = torch.argmax(pc_logits, dim=-1)
            pc_result = polarity_id_to_name[pc_predictions[0]]

            sentence['annotation'].append([pair, pc_result])
                

    return data

### category : DeBERTa / polarity : ELECTRA 

In [ ]:
def predict_from_korean_form_deberta(tokenizer_deberta, tokenizer_kelec, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for idx, sentence in enumerate(data):
        if idx % 10 == 0:
            print(idx, "/", len(data))

        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue
        for pair in entity_property_pair:
            

            tokenized_data_deberta = tokenizer_deberta(form, pair, padding='max_length', max_length=256, truncation=True)
            tokenized_data_kelec = tokenizer_kelec(form, pair, padding='max_length', max_length=256, truncation=True)

            input_ids_deberta = torch.tensor([tokenized_data_deberta['input_ids']]).to(device)
            attention_mask_deberta = torch.tensor([tokenized_data_deberta['attention_mask']]).to(device)

            input_ids_kelec = torch.tensor([tokenized_data_kelec['input_ids']]).to(device)
            attention_mask_kelec = torch.tensor([tokenized_data_kelec['attention_mask']]).to(device)

            with torch.no_grad():
                _, ce_logits = ce_model(input_ids_deberta, attention_mask_deberta)

            ce_predictions = torch.argmax(ce_logits, dim = -1)

            ce_result = tf_id_to_name[ce_predictions[0]]

            if ce_result == 'True':
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids_kelec, attention_mask_kelec)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])


    return data


## Inference

* 단일 모델
* 여러 모델
 - 두 개의 모델
 - 복수 개의 모델

### 한개 모델 inference 진행 ( ELECTRA )
단일 모델 path의 경로는 '모델 구축-기본 설정'에서 확인하기! 
1. test_category_extraction_model_path
2. test_polarity_classification_model_path



In [ ]:
def test_sentiment_analysis():

    tokenizer_kelec = AutoTokenizer.from_pretrained(base_model_elec)

    num_added_toks_kelec = tokenizer_kelec.add_special_tokens(special_tokens_dict)

    test_data = jsonlload(test_data_path)

    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_kelec, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = ElectraBaseClassifier_Cate_Base(len(tf_id_to_name), len(tokenizer_kelec))
    model.load_state_dict(torch.load(test_category_extraction_model_path, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    # print('F1 result: ', evaluation_f1(test_data, pred_data))
    pred_data = predict_from_korean_form_kelec(tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data))

    df = {'id' : [], 'sentence_form' : [], 'annotation' : []}

    for i in range(len(pred_data)) :
        df['id'].append(pred_data[i]['id'])
        df['sentence_form'].append(pred_data[i]['sentence_form'])
        df['annotation'].append(pred_data[i]['annotation'])
    df_pred = pd.DataFrame(df)


    with open('Inference_sample.jsonl', 'w') as file:
        for i in range( len(df_pred) ):
            tmp = str(df_pred['annotation'][i]).replace("\'", "\"").replace('None', 'null')
            file.write(  '{'+'\"id\": \"nikluge-sa-2022-test-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                .format( str(i+1).zfill(5)  ,   df_pred['sentence_form'][i], tmp ) +'}' ) 
            file.write("\n")

    return df_pred

In [ ]:
test_sentiment_analysis() 

### 두개 모델을 불러 inference 진행

* Category를 DeBERTa 가 아닌 RoBERTa로 찍고 싶다면 **"주석 처리된 roberta 파트를 주석 해제"** 하고 deberta 파트는 주석 처리를 하면 된다.

In [ ]:
#Category

#DeBERTa
test_category_extraction_model_path_deberta = '/content/drive/MyDrive/deberta_sample.pt'
#RoBERTa
test_category_extraction_model_path_roberta = '/content/drive/MyDrive/roberta_sample.pt'

#Polarity ( ELECTRA )
test_polarity_classification_model_path = '/content/drive/MyDrive/sample.pt'

In [ ]:
def test_sentiment_analysis():


    tokenizer_deberta = AutoTokenizer.from_pretrained(base_model_deberta)
    # tokenizer_roberta = AutoTokenizer.from_pretrained(base_model_roberta)
    tokenizer_kelec = AutoTokenizer.from_pretrained(base_model_elec)

    num_added_toks_deberta = tokenizer_deberta.add_special_tokens(special_tokens_dict)
    # num_added_toks_roberta = tokenizer_roberta.add_special_tokens(special_tokens_dict)
    num_added_toks_kelec = tokenizer_kelec.add_special_tokens(special_tokens_dict)

    test_data = jsonlload(test_data_path)

    entity_property_test_data_deberta, polarity_test_data_deberta = get_dataset(test_data, tokenizer_deberta, max_len_debe)
    # entity_property_test_data_roberta, polarity_test_data_roberta = get_dataset(test_data, tokenizer_roberta, max_len_robe)
    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_deberta, shuffle=True,
                                batch_size=batch_size)

    # entity_property_test_dataloader = DataLoader(entity_property_test_data_roberta, shuffle=True,
    #                             batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = DebertaBaseClassifier(len(tf_id_to_name), len(tokenizer_deberta))
    model.load_state_dict(torch.load(test_category_extraction_model_path_deberta, map_location=device))
    # model = RobertaBaseClassifier(len(tf_id_to_name), len(tokenizer_roberta))
    # model.load_state_dict(torch.load(test_category_extraction_model_path_roberta, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()


    # print('F1 result: ', evaluation_f1(test_data, pred_data))
    pred_data = predict_from_korean_form_deberta(tokenizer_deberta ,tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data))
    # pred_data = predict_from_korean_form_roberta(tokenizer_roberta ,tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data))

    df = {'id' : [], 'sentence_form' : [], 'annotation' : []}

    for i in range(len(pred_data)) :
        df['id'].append(pred_data[i]['id'])
        df['sentence_form'].append(pred_data[i]['sentence_form'])
        df['annotation'].append(pred_data[i]['annotation'])
    df_pred = pd.DataFrame(df)


    with open('Inference_sample.jsonl', 'w') as file:
        for i in range( len(df_pred) ):
            tmp = str(df_pred['annotation'][i]).replace("\'", "\"").replace('None', 'null')
            file.write(  '{'+'\"id\": \"nikluge-sa-2022-test-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                .format( str(i+1).zfill(5)  ,   df_pred['sentence_form'][i], tmp ) +'}' ) 
            file.write("\n")

    return df_pred

In [ ]:
test_sentiment_analysis()

### 여러 모델을 불러 inference 진행

* 각 모델 마다 잡아내는 문장의 annotation이 달라서 여러 모델을 불러서 inference를 진행하는 코드이다.
* 현재 아래 코드는
DeBERTa > ELECTRA(hidden_size_up) > ELECTRA > ELECTRA(dropout=0.5) > ELECTRA(Force)

Path ( test_category_path / test_polarity_path)

In [ ]:
#Category
test_category_extraction_model_path_k_up = '/content/drive/MyDrive/sample(hiddenup).pt'
test_category_extraction_model_path_k = '/content/drive/MyDrive/sample(category).pt'
test_category_extraction_model_path_gpu_lr = '/content/drive/MyDrive/sample(learningrate).pt'
test_category_extraction_model_path_deberta = '/content/drive/MyDrive/deberta_sample.pt'
test_category_extraction_model_path_roberta = '/content/drive/MyDrive/roberta_sample.pt'
test_category_extraction_model_path_origin = '/content/drive/MyDrive/sample(category).pt'

#Polarity
test_polarity_classification_model_path = '/content/drive/MyDrive/sample.pt'

In [ ]:
test_data_path = '/content/drive/MyDrive/test_sample.jsonl'
test_data_path_blank_first = '/content/Blank1.jsonl'
test_data_path_blank_second = '/content/Blank2.jsonl'
test_data_path_blank_third = '/content/Blank3.jsonl'
test_data_path_blank_fourth = '/content/Blank4.jsonl'
test_data_path_blank_fifth = '/content/Blank5.jsonl'
test_data_path_blank_sixth = '/content/Blank6.jsonl'

In [ ]:
def Win():

    print("Deberta!!")

    tokenizer_kelec = AutoTokenizer.from_pretrained(base_model_elec)
    tokenizer_deberta = AutoTokenizer.from_pretrained(base_model_deberta)
    tokenizer_roberta = AutoTokenizer.from_pretrained(base_model_roberta)

    num_added_toks_kelec = tokenizer_kelec.add_special_tokens(special_tokens_dict)
    num_added_toks_deberta = tokenizer_deberta.add_special_tokens(special_tokens_dict)
    num_added_toks_roberta = tokenizer_roberta.add_special_tokens(special_tokens_dict)

    test_data = jsonlload(test_data_path)

    entity_property_test_data_deberta, polarity_test_data_deberta = get_dataset(test_data, tokenizer_deberta, max_len_debe)
    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_deberta, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = DebertaBaseClassifier(len(tf_id_to_name), len(tokenizer_deberta))
    model.load_state_dict(torch.load(test_category_extraction_model_path_deberta, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form_deberta(tokenizer_deberta ,tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data))
    df_pred_first = pd.DataFrame(pred_data)

    with open('Blank1.jsonl', 'w') as file:
        for i in range( len(df_pred_first) ):
            if len(df_pred_first['annotation'][i]) == 0 :
                tmp = '[["제품 전체#일반", [null, 0, 0], "positive"]]'
                file.write(  '{'+'\"id\": \"{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( df_pred_first['id'][i]  ,   df_pred_first['sentence_form'][i], tmp ) +'}' ) 
                file.write("\n")

    print("K up!!")
    
    test_data_blank_first = jsonlload(test_data_path_blank_first)

    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data_blank_first, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_kelec, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = ElectraBaseClassifier_Cate_hiddenup(len(tf_id_to_name), len(tokenizer_kelec))
    model.load_state_dict(torch.load(test_category_extraction_model_path_k_up, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form_kelec(tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data_blank_first))
    df_pred_second = pd.DataFrame(pred_data)

    with open('Blank2.jsonl', 'w') as file:
        for i in range( len(df_pred_second) ):
            if len(df_pred_second['annotation'][i]) == 0 :
                tmp = '[["제품 전체#일반", [null, 0, 0], "positive"]]'
                file.write(  '{'+'\"id\": \"{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( df_pred_second['id'][i]  ,   df_pred_second['sentence_form'][i], tmp ) +'}' ) 
                file.write("\n")

    print("Original!!")

    test_data_blank_second = jsonlload(test_data_path_blank_second)

    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data_blank_second, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_kelec, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = ElectraBaseClassifier_Cate_Base(len(tf_id_to_name), len(tokenizer_kelec))
    model.load_state_dict(torch.load(test_category_extraction_model_path_origin, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form_kelec(tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data_blank_second))
    df_pred_third = pd.DataFrame(pred_data)

    with open('Blank3.jsonl', 'w') as file:
        for i in range( len(df_pred_third) ):
            if len(df_pred_third['annotation'][i]) == 0 :
                tmp = '[["제품 전체#일반", [null, 0, 0], "positive"]]'
                file.write(  '{'+'\"id\": \"{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( df_pred_third['id'][i]  ,   df_pred_third['sentence_form'][i], tmp ) +'}' ) 
                file.write("\n")
    
    print("K Dr!!!")

    test_data_blank_third = jsonlload(test_data_path_blank_third)

    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data_blank_third, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_kelec, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = ElectraBaseClassifier_Cate_dr05(len(tf_id_to_name), len(tokenizer_kelec))
    model.load_state_dict(torch.load(test_category_extraction_model_path_k, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form_kelec(tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data_blank_third))
    df_pred_fourth = pd.DataFrame(pred_data)

    with open('Blank4.jsonl', 'w') as file:
        for i in range( len(df_pred_fourth) ):
            if len(df_pred_fourth['annotation'][i]) == 0 :
                tmp = '[["제품 전체#일반", [null, 0, 0], "positive"]]'
                file.write(  '{'+'\"id\": \"{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( df_pred_fourth['id'][i]  ,   df_pred_fourth['sentence_form'][i], tmp ) +'}' ) 
                file.write("\n")

    print("Gpu LR!!")

    test_data_blank_fourth = jsonlload(test_data_path_blank_fourth)

    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data_blank_fourth, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_kelec, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = ElectraBaseClassifier_Cate_Base(len(tf_id_to_name), len(tokenizer_kelec))
    model.load_state_dict(torch.load(test_category_extraction_model_path_gpu_lr, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form_kelec(tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data_blank_fourth))
    df_pred_fifth = pd.DataFrame(pred_data)

    with open('Blank5.jsonl', 'w') as file:
        for i in range( len(df_pred_fifth) ):
            if len(df_pred_fifth['annotation'][i]) == 0 :
                tmp = '[["제품 전체#일반", [null, 0, 0], "positive"]]'
                file.write(  '{'+'\"id\": \"{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( df_pred_fifth['id'][i] ,   df_pred_fifth['sentence_form'][i], tmp ) +'}' ) 
                file.write("\n")

    print("The Last Forcing!!")

    test_data_final = jsonlload(test_data_path_blank_fifth)

    entity_property_test_data_kelec, polarity_test_data_kelec = get_dataset(test_data_final, tokenizer_kelec, max_len_elec)

    entity_property_test_dataloader = DataLoader(entity_property_test_data_kelec, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data_kelec, shuffle=True,
                                                  batch_size=batch_size)
    
    model = ElectraBaseClassifier_Cate_Base(len(tf_id_to_name), len(tokenizer_kelec))
    model.load_state_dict(torch.load(test_category_extraction_model_path_gpu_lr, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ElectraBaseClassifier_Pola_Base(len(polarity_id_to_name), len(tokenizer_kelec))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form_kelec_forcing(tokenizer_kelec, model, polarity_model, copy.deepcopy(test_data_final))

    df_pred_final = pd.DataFrame(pred_data)

    df_final = pd.concat([df_pred_first, df_pred_second, df_pred_third, df_pred_fourth, df_pred_fifth, df_pred_final]).sort_values(by = ['id'], axis = 0).reset_index(drop = True)

    with open('/content/drive/MyDrive/Inference_samples.jsonl', 'w') as file:
        for i in range( len(df_final) ):
            if len(df_final['annotation'][i]) != 0 :
                tmp = str(df_final['annotation'][i]).replace("\'", "\"").replace('None', 'null')
                file.write(  '{'+'\"id\": \"{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( df_final['id'][i],   df_final['sentence_form'][i], tmp ) +'}' ) 
                file.write("\n")


    return pd.DataFrame(jsonlload('/content/drive/MyDrive/Inference_samples.jsonl'))

In [ ]:
Win() # Deberta Kup Original Kdr GPU Forcing

## Counting num of Blank

In [ ]:
a = pd.DataFrame(jsonlload('/content/sample.jsonl'))
count = 0
for i in range(len(a)) :
    if len(a['annotation'][i]) == 0 :
        count += 1

count